In [ ]:
!pip install --force-reinstall numpy tensorflow==2.16.1 scikit-learn==1.5.0 pandas scikeras==0.13.0

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached tensorflow-2.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
  Using cached scikit_learn-1.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached scikeras-0.13.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_

In [ ]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [ ]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.2.3', '2.16.1', '1.5.0', '0.13.0')

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k
from sklearn.model_selection import GridSearchCV

In [ ]:
X = pd.read_csv('/content/entradas_breast.csv')
y = pd.read_csv('/content/saidas_breast.csv')

In [ ]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([
     tf.keras.layers.InputLayer(shape = (30,)),
     tf.keras.layers.Dense(units = 16, activation= 'relu', kernel_initializer= 'random_uniform'),
     tf.keras.layers.Dropout(rate = 0.2),
     tf.keras.layers.Dense(units = 16, activation= 'relu', kernel_initializer= 'random_uniform'),
     tf.keras.layers.Dropout(rate = 0.2),
     tf.keras.layers.Dense(units= 1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

In [ ]:
rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)

In [ ]:
resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

In [ ]:
resultados.mean(), resultados.std()

(0.8893483709273182, 0.034098340756648526)

In [ ]:
def criar_rede(optimizer, loss, kernel_initalizer, activation, neurons):
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape = (30,)),
      tf.keras.layers.Dense(units = neurons, activation = activation, kernel_initializer= kernel_initalizer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units = neurons, activation = activation, kernel_initializer= kernel_initalizer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units= 1, activation = 'sigmoid')])
  rede_neural.compile(optimizer = optimizer, loss = loss, metrics = ['binary_accuracy'])
  return rede_neural

In [ ]:
rede_neural = KerasClassifier(model = criar_rede)

In [ ]:
param = {
    'batch_size': [10, 30],
    'epochs': [50],
    'model__optimizer': ['adam', 'sgd', 'AdamW'],
    'model__loss' : ['binary_crossentropy'],
    'model__kernel_initalizer': ['random_uniform', 'normal'],
    'model__activation': ['relu'],
    'model__neurons': [16, 8, 4]
}

In [ ]:
grid_search = GridSearchCV(estimator = rede_neural, param_grid = param,
                          scoring = 'accuracy', cv = 5)

In [ ]:
grid_search = grid_search.fit(X, y)

In [ ]:
melhores_parametros = grid_search.best_params_
print(melhores_parametros)

{'batch_size': 10, 'epochs': 50, 'model__activation': 'relu', 'model__kernel_initalizer': 'normal', 'model__loss': 'binary_crossentropy', 'model__neurons': 8, 'model__optimizer': 'AdamW'}


In [ ]:
melhor_precisao = grid_search.best_score_
print(melhor_precisao)

0.8980903586399627


# Tentativa de Melhorias

1) 3 camadas ocultas:
      

*   A primeira com 16 neuronios e 0.3 de dropout
*   As 2 seguintes com 8 neuronios e sem droupout




In [ ]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([

     tf.keras.layers.InputLayer(shape = (30,)),

     tf.keras.layers.Dense(units = 16, activation= 'relu', kernel_initializer= 'normal'),
     tf.keras.layers.Dropout(rate = 0.3),

     tf.keras.layers.Dense(units = 8, activation= 'relu', kernel_initializer= 'normal'),


     tf.keras.layers.Dense(units = 8, activation= 'relu', kernel_initializer= 'normal'),


     tf.keras.layers.Dense(units= 1, activation = 'sigmoid')])

  otimizador = tf.keras.optimizers.AdamW(learning_rate = 0.001)

  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

rede_neural = KerasClassifier(model = criar_rede, epochs = 50, batch_size = 10, verbose=0)

resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 5, scoring = 'accuracy')

resultados.mean(), resultados.std()

(0.8927806241266882, 0.028013830219894873)

2) 3 camadas ocultas:
      

*   A primeira com 16 neuronios e 0.3 de dropout
*   As 2 seguintes com 8 neuronios e sem droupout
*   Adicionando weight_decay=0.004 no otimizador




In [ ]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([

     tf.keras.layers.InputLayer(shape = (30,)),

     tf.keras.layers.Dense(units = 16, activation= 'relu', kernel_initializer= 'normal'),
     tf.keras.layers.Dropout(rate = 0.3),

     tf.keras.layers.Dense(units = 8, activation= 'relu', kernel_initializer= 'normal'),


     tf.keras.layers.Dense(units = 8, activation= 'relu', kernel_initializer= 'normal'),


     tf.keras.layers.Dense(units= 1, activation = 'sigmoid')])

  otimizador = tf.keras.optimizers.AdamW(learning_rate = 0.001, weight_decay=0.004)

  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

rede_neural = KerasClassifier(model = criar_rede, epochs = 50, batch_size = 10, verbose=0)

resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 5, scoring = 'accuracy')

resultados.mean(), resultados.std()

(0.8910728147803135, 0.022460761042019897)

2) 3 camadas ocultas:
      

*   A primeira com 16 neuronios e 0.3 de dropout
*   As 2 seguintes com 8 neuronios e sem droupout
*   Adicionando weight_decay=0.004 no otimizador
*   Epochs = 500, batch_size = 25, cv = 10






In [ ]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([

     tf.keras.layers.InputLayer(shape = (30,)),

     tf.keras.layers.Dense(units = 16, activation= 'relu', kernel_initializer= 'normal'),
     tf.keras.layers.Dropout(rate = 0.3),

     tf.keras.layers.Dense(units = 8, activation= 'relu', kernel_initializer= 'normal'),


     tf.keras.layers.Dense(units = 8, activation= 'relu', kernel_initializer= 'normal'),


     tf.keras.layers.Dense(units= 1, activation = 'sigmoid')])

  otimizador = tf.keras.optimizers.AdamW(learning_rate = 0.001, weight_decay=0.004)

  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

rede_neural = KerasClassifier(model = criar_rede, epochs = 500, batch_size = 25, verbose=0)

resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

resultados.mean(), resultados.std()

(0.9033208020050123, 0.032641704941713354)

4) 3 camadas ocultas:
      

*   A primeira com 32 neuronios e 0.2 de dropout
*   A segunda com 16 neuronios e 0.2 de dropout
*   A terceira com 8 neuronios e 0.2 de dropout
*   Adicionando weight_decay=0.004 no otimizador
*   Epochs = 500, batch_size = 25, cv = 10




In [ ]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([

     tf.keras.layers.InputLayer(shape = (30,)),

     tf.keras.layers.Dense(units = 32, activation= 'relu', kernel_initializer= 'random_uniform'),
     tf.keras.layers.Dropout(rate = 0.2),

     tf.keras.layers.Dense(units = 16, activation= 'relu', kernel_initializer= 'normal'),
     tf.keras.layers.Dropout(rate = 0.2),

     tf.keras.layers.Dense(units = 8, activation= 'relu', kernel_initializer= 'normal'),
     tf.keras.layers.Dropout(rate = 0.2),

     tf.keras.layers.Dense(units= 1, activation = 'sigmoid')])

  otimizador = tf.keras.optimizers.AdamW(learning_rate = 0.001, decay = 0.0001, clipvalue = 0.5)

  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

rede_neural = KerasClassifier(model = criar_rede, epochs = 500, batch_size = 25, verbose=0)

resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

resultados.mean(), resultados.std()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warni

(0.9086779448621553, 0.02174792534403556)